# Modelos 4: Tentando algo diferente
**Nomes**: Bruna Guedes Pereira, Laura Medeiros Dal Ponte, Mariana Melo Pereira

## Introdução

O aprendizado de máquina consolidou-se como uma das ferramentas mais influentes da ciência contemporânea, permitindo que dados complexos promovam decisões automatizadas. Dentro desse campo, os algoritmos de classificação ocupam papel central, pois possibilitam identificar padrões e organizar informações em categorias de forma precisa e eficiente.

Métodos clássicos como Support Vector Machine (SVM), Naive Bayes e Discriminant Analysis possuem relevância histórica na resolução de problemas de classificação, sendo ainda muito utilizados em diversos experimentos científicos. Explorar o funcionamento interno desses algoritmos, compreendendo suas bases matemáticas e avaliando seu desempenho em conjuntos de dados reais, é um exercício essencial de formação acadêmica.

Dessa forma, o presente trabalho dedica-se a investigar o algoritmos ------------, implementá-lo de maneira didática e comparar seus resultados a um modelo de referência.

## Resultados e Discussões

Ao aplicar o algoritmo escolhido em um conjunto de dados real, foi possível observar seu desempenho em comparação com um classificador baseline, revelando aspectos importantes sobre sua capacidade de generalização e sensibilidade às características dos dados. A implementação em Python puro permitiu compreender detalhadamente cada etapa do processo de classificação, desde o pré-processamento até a tomada de decisão. Os resultados mostraram que, embora o modelo baseline ofereça uma referência útil, o algoritmo estudado apresentou maior precisão e melhor separação entre as classes, especialmente em cenários com dados bem distribuídos. Essa análise reforça a importância de entender o funcionamento interno dos algoritmos para fazer escolhas mais informadas em projetos de aprendizado de máquina.

## Conclusões

Este notebook teve como objetivo compreender e aplicar de forma didática um algoritmo de classificação supervisionada, destacando tanto sua fundamentação teórica quanto sua implementação prática em Python puro. Ao induzir um modelo com o algoritmo escolhido e compará-lo com um classificador baseline, foi possível avaliar seu desempenho e entender suas vantagens e limitações em cenários reais. A construção manual das funções fortaleceu o aprendizado sobre os mecanismos internos do algoritmo, promovendo autonomia na exploração de novas técnicas de aprendizado de máquina. Assim, o material contribui para o desenvolvimento de uma habilidade essencial: a capacidade de estudar e aplicar algoritmos não abordados diretamente em sala de aula.


## Referências Bibliográficas
1. IBM. Máquina de vetores de suporte. Disponível em: <https://www.ibm.com/br-pt/think/topics/support-vector-machine>.
2. OPENAI. ChatGPT. São Francisco: OpenAI, 2025. Disponível em: <https://chat.openai.com/>. Acesso em: 10 set. 2025.